# Motion segmentation

In [1]:
# For Kmeans algorithm. See PGM otherwise for handcrafted class
from sklearn.cluster import KMeans
# For loading matlab matrix file
from scipy.io import loadmat
# Hungarian algorithm
from scipy.optimize import linear_sum_assignment

#from scipy.sparse.linalg import eigs, eigsh
from scipy.stats import itemfreq
import numpy as np

#Progressbar
from tqdm import tqdm

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
%precision %.5f
%load_ext line_profiler

plt.rc('text', usetex=True)
plt.rc('font',**{'family':'serif','serif':['Palatino']})

In [4]:
#Algorithm and error
from error_evaluation import *
from ksubspaces import *
from spectral_clustering import *
from SSC import *

Extrait de 

@article{elhamifar2013sparse,

  title={Sparse subspace clustering: Algorithm, theory, and applications},
  
  author={Elhamifar, Ehsan and Vidal, Rene},
  
  journal={IEEE transactions on pattern analysis and machine intelligence},
  
  volume={35},
  
  number={11},
  
  pages={2765--2781},
  
  year={2013},
  
  publisher={IEEE}
  
}


![article](real_data_article.png)

## Globals

In [5]:
DATA_DIR = "data/Hopkins155/"
NEEDLE = "_truth.mat"

## Data

In [6]:
import glob
import os

In [7]:
motion_data = {}
count_files = 0
# iglob for iterator
## Recursive=True to use **
for file in glob.iglob(DATA_DIR + "**/*" + NEEDLE, recursive = True):
    mat_name = file.split("/")[-1].split(NEEDLE)[0]
    try:
        # Load matlab matrix
        motion_data[mat_name] = loadmat(file)
        # Cast in float64 x
        motion_data[mat_name]['x'] = motion_data[mat_name]\
        ['x'].astype(np.float64, subok = True, copy = False)
        # Cast number of points in int
        motion_data[mat_name]['points'] = motion_data[mat_name]\
        ['points'][0, 0].astype(int)
        # Cast number of frames in int
        motion_data[mat_name]['frames'] = motion_data[mat_name]\
        ['frames'][0, 0].astype(int)
        # Substract 1 to matlab labels
        motion_data[mat_name]['s'] -= 1
    except OSError:
        print ('cannonot open ', file)
    else:
        count_files += 1
        motion_data[mat_name]["data"] = \
        np.transpose(motion_data[mat_name]['x'][:2, : :],
                     axes = (0, 2, 1)).reshape(2 * motion_data[mat_name]['frames'],
                                               motion_data[mat_name]["points"])
print ("Number of files loaded : {}".format(count_files))

Number of files loaded : 156


## Results zone : SSC

In [10]:
Res = np.load('motion_res_SSC.npy').item()

### $tau = 1e3$, $mu=1e3$

In [38]:
tau = 1e3
mu = 1e3

In [44]:
Res[(tau, mu)] = {}
for key in tqdm(motion_data.keys()):
    # Number of ground_truth classes
    nbr_classes = motion_data[key]["s"].max() + 1
    Res[(tau, mu)][key] = evaluate_error(SSC(motion_data[key]["data"], nbr_classes, tau, mu)[1],
                              motion_data[key]["s"].reshape(-1))

In [58]:
Res.keys()

dict_keys([(1000.0, 1000.0), (10000.0, 1000.0)])

In [64]:
Res[(1e4, 1e3)]

0.01303

### $tau = 1e4$, $mu=1e3$

In [65]:
tau = 1e4
mu = 1e3
Res[(tau, mu)] = {}
for key in tqdm(motion_data.keys()):
    # Number of ground_truth classes
    nbr_classes = motion_data[key]["s"].max() + 1
    Res[(tau, mu)][key] = evaluate_error(SSC(motion_data[key]["data"], nbr_classes, tau, mu)[1],
                              motion_data[key]["s"].reshape(-1))

100%|██████████| 156/156 [24:43<00:00,  9.51s/it]


### $tau = 1e5$, $mu=1e3$

In [68]:
tau = 1e5
mu = 1e3
Res[(tau, mu)] = {}
for key in tqdm(motion_data.keys()):
    # Number of ground_truth classes
    nbr_classes = motion_data[key]["s"].max() + 1
    Res[(tau, mu)][key] = evaluate_error(SSC(motion_data[key]["data"], nbr_classes, tau, mu)[1],
                              motion_data[key]["s"].reshape(-1))

100%|██████████| 156/156 [23:49<00:00,  9.16s/it]


### $tau = 1e6$, $mu=1e3$

In [82]:
tau = 1e6
mu = 1e3
Res[(tau, mu)] = {}
for key in tqdm(motion_data.keys()):
    # Number of ground_truth classes
    nbr_classes = motion_data[key]["s"].max() + 1
    Res[(tau, mu)][key] = evaluate_error(SSC(motion_data[key]["data"], nbr_classes, tau, mu)[1],
                              motion_data[key]["s"].reshape(-1))

100%|██████████| 156/156 [19:55<00:00,  7.66s/it]


### $tau = 1e7$, $mu=1e3$

In [92]:
tau = 1e7
mu = 1e3
Res[(tau, mu)] = {}
for key in tqdm(motion_data.keys()):
    # Number of ground_truth classes
    nbr_classes = motion_data[key]["s"].max() + 1
    Res[(tau, mu)][key] = evaluate_error(SSC(motion_data[key]["data"], nbr_classes, tau, mu)[1],
                              motion_data[key]["s"].reshape(-1))

100%|██████████| 156/156 [32:19<00:00, 12.43s/it]


In [11]:
#np.save('motion_res_SSC.npy', Res)

## Results zone : K-subspace
Figure 11.4 shows the singular values of the matrix of feature point trajectories
of a single motion for several videos in the data set. Note that the singular value
curve has a knee around 4, corroborating the approximate 4-dimensionality of the
motion data in each video.

In [20]:
K_sub = {}

In [ ]:
for restarts in [500]:
    K_sub[restarts] = {}
    for key in tqdm(motion_data.keys()):
        # Number of ground_truth classes
        nbr_classes = motion_data[key]["s"].max() + 1
        K_sub[restarts][key] = evaluate_error(ksubspaces(motion_data[key]["data"], nbr_classes,
                                                         nbr_classes * [4], restarts)[0][0],
                                              motion_data[key]["s"].reshape(-1))
np.save('motion_res_ksub.npy', K_sub)

  1%|▏         | 2/156 [00:21<27:11, 10.60s/it]

# Test zone

## Cars1

|$\tau$ | $\mu_2$| $error (\%)$|
|-------|--------|-------------|
|1e6    |1e6     |1.30         |
|1e8    |1e6     |16.94        |
|1e3    |1e6     |0.00         |
|1e3    |1e4     |0.32         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |
|    |     |         |


In [13]:
key = "cars1"

In [14]:
# Data
motion_data[key]["data"].shape

(40, 307)

In [15]:
# Labels
print ("Number of groups : {}".format(motion_data[key]["s"].max() + 1))
motion_data[key]["s"].shape

Number of groups : 2


(307, 1)

In [73]:
T = SSC(motion_data["cars1"]["data"], 2, 10e3, 10e3)

In [76]:
%lprun -f compute_sparse_C T = SSC(motion_data["cars1"]["data"], 2, 10e3, 10e3)

In [75]:
# 10e5 10e5 : 0.01303
evaluate_error(T[1], motion_data["cars1"]["s"].reshape(-1))

0.01303

## 2RT3RC

|$\tau$ | $\mu_2$| $error (\%)$|
|-------|--------|-------------|
|1e3    |1e6     |48.46         |
|1e5    |1e6     |23.41         |
|1e5    |1e3     |2.36         |
|1e2    |     |    Not converging     |
|1e6    |1e1     |40.47         |
|1e6    |1e3     |1.63         |
|1e6    |1e4     |1.63         |
|1e6    |1e5     |2.90       |
|1e6    |1e8    |35.57         |
|1e7    |1e3     |1.63         |
|    |     |         |

In [96]:
key = "2RT3RC"
T = SSC(motion_data[key]["data"], motion_data[key]["s"].max() + 1, 1e6, 1e3)
evaluate_error(T[1], motion_data[key]["s"].reshape(-1))

[[ 3.08478096  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          4.51525604  0.         ...,  0.          0.          0.        ]
 [ 0.          0.          2.44128282 ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  7.40659508  0.          0.        ]
 [ 0.          0.          0.         ...,  0.          1.58260061  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.99505101]]
Erreur : 0.016333938294010888


0.01633

In [98]:
%lprun -f  compute_sparse_C T = SSC(motion_data[key]["data"], motion_data[key]["s"].max() + 1, 1e7, 1e3)

[[  3.14557565   0.           0.         ...,   0.           0.           0.        ]
 [  0.           8.01983806   0.         ...,   0.           0.           0.        ]
 [  0.           0.           2.78783241 ...,   0.           0.           0.        ]
 ..., 
 [  0.           0.           0.         ...,  16.18052198   0.           0.        ]
 [  0.           0.           0.         ...,   0.           1.31183943
    0.        ]
 [  0.           0.           0.         ...,   0.           0.
    3.39565212]]


## K-subspaces

In [19]:
key = "2RT3RC"
nbr_subspaces = motion_data[key]["s"].max() + 1
T = ksubspaces(motion_data[key]["data"], nbr_subspaces , nbr_subspaces * [5] , 3, verbose = True)
# evaluate_error(T[1], motion_data[key]["s"].reshape(-1))

Current error for 0 replicate : 3.54615e+02
Current error for 1 replicate : 2.32741e+02
Current error for 2 replicate : 1.83868e+02


In [17]:
motion_data[key]["s"].reshape(-1)

array([1, 2, 0, 1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2, 0, 0, 2, 1, 1, 1, 1, 2,
       1, 0, 1, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 2, 1, 1,
       0, 1, 1, 2, 1, 1, 0, 1, 1, 0, 2, 1, 0, 2, 1, 2, 1, 1, 1, 0, 2, 2, 0,
       1, 0, 2, 0, 0, 0, 0, 1, 0, 1, 2, 2, 1, 1, 0, 2, 2, 0, 0, 2, 0, 2, 1,
       2, 1, 2, 2, 1, 2, 2, 1, 0, 0, 2, 1, 1, 1, 2, 2, 2, 2, 0, 0, 1, 2, 0,
       2, 2, 1, 1, 0, 0, 2, 2, 1, 0, 2, 1, 0, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1,
       1, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 2, 0, 1, 1, 1, 2, 0, 0, 2, 2, 0, 2,
       0, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2,
       2, 1, 1, 1, 0, 1, 1, 1, 2, 2, 1, 2, 0, 1, 2, 1, 0, 1, 2, 2, 2, 2, 2,
       1, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2,
       1, 0, 2, 2, 1, 0, 2, 2, 2, 0, 1, 2, 1, 2, 0, 2, 2, 1, 2, 2, 1, 0, 2,
       0, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 2, 2, 0, 2, 0, 1, 1, 2, 2, 0,
       2, 2,

In [105]:
data = motion_data[key]["data"]